In [ ]:
# IMPORTS
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model

In [ ]:
# GLOBAL VARIABLES
SIZE = (28,28)
CLASSES = dict(zip(range(0,25),list(map(chr, range(97, 123)))))

In [ ]:
def get_data(path):
    data = np.genfromtxt(path, delimiter=',', skip_header=1)
    labels = data[0:, 0].astype(np.uint8)
    data = np.array([np.reshape(i[1:], SIZE) for i in data.astype(np.uint8)])
    return data, labels

In [ ]:
# DATA
train_images, train_labels = get_data('data/sign_mnist_train.csv') # get train data
test_images, test_labels = get_data('data/sign_mnist_test.csv') # get test data

In [ ]:
# DATA PREPROCESSING
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
def show_image(mat, label):
    plt.xlabel(CLASSES[label], color='black')
    plt.imshow(mat, cmap=plt.cm.binary)
    plt.show()

for i in range(2):
    show_image(train_images[i], train_labels[i])

In [ ]:
# prepare the model
model_params = [
    keras.layers.Flatten(input_shape=SIZE),  # input layer  (1)
    keras.layers.Dense(128, activation='relu'),  # hidden layer (2)
    keras.layers.Dense(128, activation='relu'),  # hidden layer (3)
    keras.layers.Dense(26, activation='softmax') # output layer (4)
]
model = keras.Sequential(model_params)
model.summary()  # let's have a look at our model so far
# compile the model with 
model.compile(optimizer='adam', # optimization fn
              loss='sparse_categorical_crossentropy', # loss fn
              metrics=['accuracy'])

In [ ]:
# train the model with the train data
model.fit(train_images, train_labels, epochs=20)
# Evaluate the model
loss, accuracy = model.evaluate(test_images,  test_labels, verbose=0)
format_ = lambda num: round(num*100, 3) # formats the number
print(f"Accuracy : {format_(accuracy)}%\nLoss : {format_(loss)}%")

In [ ]:
if not os.path.exists("models"): os.mkdir("models")
model.save("models/model.h5")

In [ ]:
'''
returns an array of predictions
shape -> (prediction data len, no of classes)
each index of the inner array pointing to the index of the CLASS variable
'''
predictions = model.predict(test_images) # make prediction

In [ ]:
# check the accuracy of the model
from termcolor import colored
def predict(model, batch=10, verbose=False):
    loss = 0
    if batch > len(test_images):
        print(f"DATA SIZE ({len(test_images)}) EXCEEDED! ")
        return
    for i in range(batch):
        prediction = model.predict(np.array([test_images[i]]))
        predicted_class, actual_class = CLASSES[np.argmax(prediction)], CLASSES[test_labels[i]]
        # checks for the classes and sets color's depending on that
        color = 'green' if predicted_class == actual_class else 'red'
        if color == 'red': loss += 1
        if verbose: print(colored(f'[{i}] Predicted : {predicted_class : <2}|  Expected : {actual_class}', color))
    perc = round(100/(batch/(batch-loss)), 3)
    print(f"Accuracy {perc}%\nLoss : {100-perc}%")

In [ ]:
predict(model)